In [36]:
import os
from dotenv import load_dotenv
from pathlib import Path
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

# sklearn imports
import sklearn.preprocessing as preprocessing
import sklearn.model_selection as model_selection
import sklearn.linear_model as linear_model
import sklearn.metrics as metrics

# imblearn imports
import imblearn.ensemble as ensemble
import imblearn.metrics as imblearn_metrics

In [37]:
# Functions

def increase_dummies(x):
    if x > 0:
        return 1
    else:
        return 0
    
def descending_sort_features(features, importances):
    indices = np.argsort(importances)
    
    sorted_importances = importances[indices[::-1]]
    sorted_features = features[indices[::-1]]
    
    result = np.dstack((sorted_features, sorted_importances))[0].tolist()
    
    return result

In [38]:
# read in saved csv

raw_path = Path('data/raw_nyt_watson.csv')
indices_path = Path('data/global_indicies.csv')

raw_df = pd.read_csv(raw_path, parse_dates=True, index_col='Date')
indices_df = pd.read_csv(indices_path)

In [39]:
# set indicies_df date index

indices_df["Date"] = pd.to_datetime(indices_df["Date"]).dt.date
indices_df = indices_df.set_index("Date")

In [40]:
# Pivot on Ticker

indices_pivot = indices_df.pivot(columns = "Ticker", values = "Close")

In [41]:
# Forward fill on any na in indices

indices_pivot = indices_pivot.fillna(method="ffill")

In [42]:
# Rename columns so its clear that these are closes

d = {
    'CAC' : 'CAC Close',
    'FTSE' : 'FTSE Close',
    'MDAX' : 'MDAX Close',
    'SP500' : 'SP500 Close',
    'TOPIX' : 'TOPIX Close'
}

indices_pivot = indices_pivot.rename(columns=d)

In [43]:
# Calculate percent change for each index

indices_pivot.insert(0, "TOPIX percent change", indices_pivot['TOPIX Close'].pct_change() * 100)
indices_pivot.insert(0, "SP500 percent change", indices_pivot['SP500 Close'].pct_change() * 100)
indices_pivot.insert(0, "MDAX percent change", indices_pivot['MDAX Close'].pct_change() * 100)
indices_pivot.insert(0, "FTSE percent change", indices_pivot['FTSE Close'].pct_change() * 100)
indices_pivot.insert(0, "CAC percent change", indices_pivot['CAC Close'].pct_change() * 100)

In [44]:
# create copy of read-in features

sentiment_df = raw_df.copy()

In [45]:
# drop unneccesary information

sentiment_df = sentiment_df.drop(columns=['Source', 'Headline', 'Lead Paragraph', 'URL'])

In [46]:
# get dummies

sentiment_df = pd.get_dummies(sentiment_df)

In [47]:
# group by date and take the mean of the sentiment scores

mean_sentiment_df = sentiment_df.groupby(['Date']).mean()

In [48]:
# concatenate the features and target data

mean_sentiment_df = pd.concat([indices_pivot.loc[:, "CAC percent change":"TOPIX percent change"], mean_sentiment_df], axis=1)

In [49]:
# create categories for the percent increase, 1 for increase, 0 for decrease/no change

mean_sentiment_df.insert(0, "TOPIX increase", mean_sentiment_df['TOPIX percent change'].apply(increase_dummies))
mean_sentiment_df.insert(0, "SP500 increase", mean_sentiment_df['SP500 percent change'].apply(increase_dummies))
mean_sentiment_df.insert(0, "MDAX increase", mean_sentiment_df['MDAX percent change'].apply(increase_dummies))
mean_sentiment_df.insert(0, "FTSE increase", mean_sentiment_df['FTSE percent change'].apply(increase_dummies))
mean_sentiment_df.insert(0, "CAC increase", mean_sentiment_df['CAC percent change'].apply(increase_dummies))

# create categories for the sentiment, 1 for positive, 0 for negative/neutral
mean_sentiment_df['Headline Sentiment'] = mean_sentiment_df["Headline Sentiment"].apply(increase_dummies)
mean_sentiment_df['Lead Paragraph Sentiment'] = mean_sentiment_df["Lead Paragraph Sentiment"].apply(increase_dummies)

In [50]:
# Drop na

mean_sentiment_df.dropna(inplace=True)

In [51]:
pct_columns = ["CAC percent change", "FTSE percent change", "MDAX percent change", "SP500 percent change", "TOPIX percent change"]
inc_columns = ["CAC increase", "FTSE increase", "MDAX increase", "SP500 increase", "TOPIX increase"]

# create dataframe for BalancedRandomForest
brf_df = mean_sentiment_df.drop(pct_columns, axis=1)

In [53]:
x_cols = [i for i in brf_df.columns if i not in inc_columns]

In [54]:
X = brf_df[x_cols]

brf_indices = {
    "CAC" : {},
    "FTSE" : {}, 
    "MDAX" : {}, 
    "SP500" : {}, 
    "TOPIX" : {},
}

brf_features = {}

for key in brf_indices.keys():
    brf_indices[key]["target"] = brf_df[key + " increase"]

In [58]:
# define scaler, split percentage, and random state
scaler = StandardScaler()
train_size = 0.7
rs = 1

In [59]:
brf_features["X train"], brf_features["X test"] = train_test_split(X, train_size=train_size, random_state=rs)

scaler.fit(brf_features["X train"])

brf_features["X train scaled"] = scaler.transform(brf_features["X train"])
brf_features["X test scaled"] = scaler.transform(brf_features["X test"])

brf = ensemble.BalancedRandomForestClassifier(n_estimators=10000, random_state=rs)

for key, brf_data in brf_indices.items():
    
    print("test 1")
    
    brf_data["y train"], brf_data["y test"] = train_test_split(brf_data["target"], train_size=train_size, random_state=rs)
    print("test 2")
    brf.fit(brf_features["X train scaled"], brf_data["y train"])
    
    print("test 3")
    
    brf_data["predictions"] = brf.predict(brf_features["X test scaled"])
    
    brf_data["Accuracy"] = metrics.balanced_accuracy_score(brf_data["y test"], brf_data["predictions"]) 
    
    brf_data["Confusion Matrix"] = metrics.confusion_matrix(brf_data["y test"], brf_data["predictions"])
    
    brf_data["Classification Report"] = imblearn_metrics.classification_report_imbalanced(
        brf_data["y test"], 
        brf_data["predictions"], 
        digits=4
    )
    
    brf_data["Feature Importance"] = descending_sort_features(
        brf_features["X train"].columns, 
        brf.feature_importances_
    )
    
del brf


test 1
test 2
test 3
test 1
test 2
test 3
test 1
test 2
test 3
test 1
test 2
test 3
test 1
test 2
test 3


In [60]:
for key, value in brf_indices.items():
    print(f"{key} Accuracy: {value['Accuracy']:.4f}\n")
    print(f"{key} Confusion Matrix: \n{value['Confusion Matrix']}\n")
    print(f"{key} Classification Report: \n{value['Classification Report']}\n")
    print(f"{key} Top Five features")
    
    for i in range(5):
        print(f"{value['Feature Importance'][i][0]}: {value['Feature Importance'][i][1]:.4f}")
        
    print("\n")

CAC Accuracy: 0.5343

CAC Confusion Matrix: 
[[22 13]
 [28 22]]

CAC Classification Report: 
                   pre       rec       spe        f1       geo       iba       sup

          0     0.4400    0.6286    0.4400    0.5176    0.5259    0.2818        35
          1     0.6286    0.4400    0.6286    0.5176    0.5259    0.2714        50

avg / total     0.5509    0.5176    0.5509    0.5176    0.5259    0.2757        85


CAC Top Five features
Headline Anger: 0.0853
Lead Paragraph Sadness: 0.0829
Headline Joy: 0.0814
Lead Paragraph Anger: 0.0793
Lead Paragraph Fear: 0.0769


FTSE Accuracy: 0.4338

FTSE Confusion Matrix: 
[[19 18]
 [31 17]]

FTSE Classification Report: 
                   pre       rec       spe        f1       geo       iba       sup

          0     0.3800    0.5135    0.3542    0.4368    0.4265    0.1848        37
          1     0.4857    0.3542    0.5135    0.4096    0.4265    0.1790        48

avg / total     0.4397    0.4235    0.4442    0.4215    0.4265    0.